In [1]:
# ==================== 生态相似性迁移实验 ====================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("生态相似性迁移实验")
print("="*80)

# 生态区信息
ecoregion_info = {
    'I01': {'group': 'A', 'climate': 'Cold temperate', 'forest_type': 'Deciduous coniferous'},
    'I02': {'group': 'B', 'climate': 'Mid temperate', 'forest_type': 'Mixed forest'},
    'I03': {'group': 'B', 'climate': 'Mid temperate', 'forest_type': 'Others'},
    'I04': {'group': 'B', 'climate': 'Mid temperate', 'forest_type': 'Mixed forest'},
    'I05': {'group': 'B', 'climate': 'Mid temperate', 'forest_type': 'Agroforestry'},
    'I07': {'group': 'C', 'climate': 'Warm temperate', 'forest_type': 'Deciduous broadleaf'},
    'I10': {'group': 'C', 'climate': 'Warm temperate', 'forest_type': 'Deciduous broadleaf'},
    'I12': {'group': 'C', 'climate': 'Warm temperate', 'forest_type': 'Others'},
    'I14': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I15': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Mixed forest'},
    'I17': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Agroforestry'},
    'I19': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Agroforestry'},
    'I20': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I21': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I22': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I23': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I24': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I25': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I26': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I27': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I28': {'group': 'D', 'climate': 'Subtropical', 'forest_type': 'Evergreen broadleaf'},
    'I30': {'group': 'E', 'climate': 'Tropical', 'forest_type': 'Evergreen broadleaf'},
    'I31': {'group': 'E', 'climate': 'Tropical', 'forest_type': 'Evergreen broadleaf'},
    'II01': {'group': 'F', 'climate': 'Temperate continental', 'forest_type': 'Others'},
    'II05': {'group': 'F', 'climate': 'Cold/mid temperate', 'forest_type': 'Evergreen coniferous'},
    'II07': {'group': 'F', 'climate': 'Mid temperate continental', 'forest_type': 'Evergreen coniferous'},
    'III07': {'group': 'F', 'climate': 'Plateau cold temperate', 'forest_type': 'Evergreen coniferous'},
    'III09': {'group': 'F', 'climate': 'Plateau tropical monsoon', 'forest_type': 'Evergreen broadleaf'}
}

class EcoMigrationExperiment:
    """生态相似性迁移实验类"""

    def __init__(self, df, feature_cols, target_col='b1', random_state=42):
        self.df = df
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.random_state = random_state
        self.models = {}
        self.scalers = {}
        self.test_data = {}
        self.performance = {}

        # 分组信息
        self.groups = {
            'B': ['I02', 'I03', 'I04', 'I05'],
            'C': ['I07', 'I10', 'I12'],
            'D': ['I14', 'I15', 'I17', 'I19', 'I20', 'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28'],
            'E': ['I30', 'I31'],
            'F': ['II01', 'II05', 'II07', 'III07', 'III09']
        }

        # 森林类型分组
        self.forest_types = {
            'Evergreen broadleaf': ['I14', 'I20', 'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28', 'I30', 'I31', 'III09'],
            'Mixed forest': ['I02', 'I04', 'I15'],
            'Deciduous broadleaf': ['I07', 'I10'],
            'Deciduous coniferous': ['I01'],
            'Evergreen coniferous': ['II05', 'II07', 'III07'],
            'Agroforestry': ['I05', 'I17', 'I19'],
            'Others': ['I03', 'I12', 'II01']
        }

    def train_local_model(self, ecoregion_id):
        """训练本地模型"""
        print(f"训练生态区 {ecoregion_id} 的本地模型...")

        # 过滤数据
        df_eco = self.df[self.df['ID'] == ecoregion_id].copy()

        if len(df_eco) < 100:
            print(f"  样本量不足 ({len(df_eco)})，跳过")
            return None

        # 分割数据
        X = df_eco[self.feature_cols]
        y = df_eco[self.target_col]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=self.random_state, stratify=y
        )

        # 标准化
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # 训练模型
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=self.random_state,
            n_jobs=-1,
            verbose=0
        )

        model.fit(X_train_scaled, y_train)

        # 本地性能
        y_pred = model.predict(X_test_scaled)
        local_acc = accuracy_score(y_test, y_pred)
        local_f1 = f1_score(y_test, y_pred)

        print(f"  本地性能: 准确率={local_acc:.4f}, F1={local_f1:.4f}, 样本量={len(X_train)}")

        # 保存结果
        self.models[ecoregion_id] = model
        self.scalers[ecoregion_id] = scaler
        self.test_data[ecoregion_id] = {
            'X': X_test_scaled,
            'y': y_test
        }
        self.performance[ecoregion_id] = {
            'accuracy': local_acc,
            'f1': local_f1,
            'train_samples': len(X_train),
            'test_samples': len(X_test)
        }

        return local_acc

    def train_all_models(self):
        """训练所有生态区的模型"""
        print("训练所有生态区的本地模型...")

        for eco_id in list(ecoregion_info.keys()):
            self.train_local_model(eco_id)

        print(f"\n完成训练 {len(self.models)} 个生态区模型")

    def migration_test(self, source_id, target_id):
        """迁移测试：用源生态区模型预测目标生态区"""
        if source_id not in self.models or target_id not in self.test_data:
            print(f"缺少模型或测试数据: source={source_id}, target={target_id}")
            return None

        # 获取源模型和标准化器
        source_model = self.models[source_id]
        source_scaler = self.scalers[source_id]

        # 获取目标测试数据
        target_X = self.test_data[target_id]['X']
        target_y = self.test_data[target_id]['y']

        # 注意：这里我们使用源标准化器对目标数据进行标准化
        # 在实际应用中，目标数据应该使用源的标准化器进行转换
        # 但由于我们之前已经用目标生态区的标准化器标准化过了，这里需要重新用源标准化器

        # 重新获取原始数据并转换
        df_target = self.df[self.df['ID'] == target_id].copy()
        X_target_original = df_target[self.feature_cols]
        y_target_original = df_target[self.target_col]

        # 使用相同的随机种子分割，确保测试集一致
        X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
            X_target_original, y_target_original,
            test_size=0.2, random_state=self.random_state, stratify=y_target_original
        )

        # 使用源标准化器进行转换
        X_test_migrated = source_scaler.transform(X_test_t)

        # 预测
        y_pred = source_model.predict(X_test_migrated)

        # 计算性能
        mig_acc = accuracy_score(y_test_t, y_pred)
        mig_f1 = f1_score(y_test_t, y_pred)

        # 本地性能（用于比较）
        local_acc = self.performance[target_id]['accuracy']

        # 性能损失
        performance_loss = (local_acc - mig_acc) / local_acc * 100

        return {
            'source': source_id,
            'target': target_id,
            'migration_accuracy': mig_acc,
            'migration_f1': mig_f1,
            'local_accuracy': local_acc,
            'performance_loss': performance_loss,
            'climate_similarity': self.get_climate_similarity(source_id, target_id),
            'forest_type_similarity': self.get_forest_type_similarity(source_id, target_id)
        }

    def get_climate_similarity(self, eco1, eco2):
        """计算气候相似性（简化版）"""
        climate1 = ecoregion_info[eco1]['climate']
        climate2 = ecoregion_info[eco2]['climate']

        # 气候相似性评分
        climate_hierarchy = {
            'Cold temperate': 1,
            'Mid temperate': 2,
            'Warm temperate': 3,
            'Subtropical': 4,
            'Tropical': 5,
            'Temperate continental': 6,
            'Cold/mid temperate': 7,
            'Mid temperate continental': 8,
            'Plateau cold temperate': 9,
            'Plateau tropical monsoon': 10
        }

        # 气候距离（越小越相似）
        distance = abs(climate_hierarchy.get(climate1, 0) - climate_hierarchy.get(climate2, 0))
        similarity = 1 / (1 + distance)  # 转换为相似度

        return similarity

    def get_forest_type_similarity(self, eco1, eco2):
        """计算森林类型相似性"""
        type1 = ecoregion_info[eco1]['forest_type']
        type2 = ecoregion_info[eco2]['forest_type']

        # 森林类型相似性矩阵（简化）
        type_similarity = {
            'Evergreen broadleaf': {'Evergreen broadleaf': 1.0, 'Mixed forest': 0.6, 'Deciduous broadleaf': 0.4},
            'Mixed forest': {'Evergreen broadleaf': 0.6, 'Mixed forest': 1.0, 'Deciduous broadleaf': 0.8},
            'Deciduous broadleaf': {'Evergreen broadleaf': 0.4, 'Mixed forest': 0.8, 'Deciduous broadleaf': 1.0},
            'Evergreen coniferous': {'Evergreen coniferous': 1.0, 'Deciduous coniferous': 0.7},
            'Deciduous coniferous': {'Evergreen coniferous': 0.7, 'Deciduous coniferous': 1.0},
            'Agroforestry': {'Agroforestry': 1.0, 'Others': 0.5},
            'Others': {'Agroforestry': 0.5, 'Others': 1.0}
        }

        # 获取相似性
        if type1 in type_similarity and type2 in type_similarity[type1]:
            return type_similarity[type1][type2]
        elif type2 in type_similarity and type1 in type_similarity[type2]:
            return type_similarity[type2][type1]
        else:
            return 0.2  # 默认较低相似性

    def experiment_1_within_group_migration(self):
        """实验1：组内迁移（B-F组）"""
        print("\n" + "="*80)
        print("实验1：组内迁移（B-F组）")
        print("="*80)

        results = []

        for group_name, eco_list in self.groups.items():
            print(f"\n组 {group_name} 内迁移实验:")

            # 只选择有模型的生态区
            available_ecos = [e for e in eco_list if e in self.models]

            if len(available_ecos) < 2:
                print(f"  组 {group_name} 可用生态区不足2个，跳过")
                continue

            # 组内所有可能的迁移对
            for i in range(len(available_ecos)):
                for j in range(len(available_ecos)):
                    if i == j:
                        continue

                    source = available_ecos[i]
                    target = available_ecos[j]

                    print(f"  迁移: {source} -> {target}")
                    result = self.migration_test(source, target)

                    if result:
                        result['experiment'] = 'Within-group'
                        result['group'] = group_name
                        results.append(result)

        return pd.DataFrame(results)

    def experiment_2_cross_group_migration(self):
        """实验2：跨组迁移（两两组合）"""
        print("\n" + "="*80)
        print("实验2：跨组迁移（两两组合）")
        print("="*80)

        results = []

        # 获取所有组
        group_names = list(self.groups.keys())

        # 每个组选择2个代表性生态区（样本量大的）
        representative_ecos = {}
        for group in group_names:
            ecos_in_group = [e for e in self.groups[group] if e in self.models]

            if len(ecos_in_group) >= 2:
                # 按样本量排序
                ecos_with_samples = []
                for eco in ecos_in_group:
                    if eco in self.performance:
                        ecos_with_samples.append((eco, self.performance[eco]['train_samples']))

                # 按样本量降序排序，取前2个
                ecos_with_samples.sort(key=lambda x: x[1], reverse=True)
                representative_ecos[group] = [e[0] for e in ecos_with_samples[:2]]
            elif len(ecos_in_group) == 1:
                representative_ecos[group] = ecos_in_group
            else:
                representative_ecos[group] = []

        # 跨组迁移（两两组合）
        for i in range(len(group_names)):
            for j in range(i+1, len(group_names)):
                group1 = group_names[i]
                group2 = group_names[j]

                print(f"\n跨组迁移: {group1} -> {group2}")

                # 获取代表生态区
                ecos_group1 = representative_ecos.get(group1, [])
                ecos_group2 = representative_ecos.get(group2, [])

                if not ecos_group1 or not ecos_group2:
                    continue

                # 所有可能的组合
                for source in ecos_group1:
                    for target in ecos_group2:
                        print(f"  迁移: {source}({group1}) -> {target}({group2})")
                        result = self.migration_test(source, target)

                        if result:
                            result['experiment'] = 'Cross-group'
                            result['source_group'] = group1
                            result['target_group'] = group2
                            results.append(result)

        return pd.DataFrame(results)

    def experiment_3_forest_type_migration(self):
        """实验3：森林类型迁移"""
        print("\n" + "="*80)
        print("实验3：森林类型迁移")
        print("="*80)

        results = []

        # 选择森林类型和样本量
        forest_type_samples = {}

        # 收集每个森林类型的生态区及其样本量
        for eco_id, info in ecoregion_info.items():
            if eco_id not in self.models:
                continue

            forest_type = info['forest_type']
            if forest_type not in forest_type_samples:
                forest_type_samples[forest_type] = []

            sample_count = self.performance[eco_id]['train_samples']
            forest_type_samples[forest_type].append((eco_id, sample_count, self.performance[eco_id]['accuracy']))

        # 对每个森林类型，按样本量和精度选择代表性生态区
        selected_ecos = {}
        for forest_type, eco_list in forest_type_samples.items():
            # 按样本量降序排序
            eco_list.sort(key=lambda x: x[1], reverse=True)

            # 选择前2个，但要考虑精度
            if len(eco_list) >= 2:
                # 如果样本量相差不大（<20%），选择精度更高的
                if eco_list[0][1] * 0.8 < eco_list[1][1]:
                    # 样本量相近，比较精度
                    if eco_list[0][2] > eco_list[1][2]:
                        selected = [eco_list[0][0], eco_list[1][0]]
                    else:
                        selected = [eco_list[1][0], eco_list[0][0]]
                else:
                    # 样本量差异大，选择样本量最大的两个
                    selected = [eco_list[0][0], eco_list[1][0]]
            elif len(eco_list) == 1:
                selected = [eco_list[0][0]]
            else:
                selected = []

            selected_ecos[forest_type] = selected

        # 森林类型迁移：相同森林类型但不同气候带的生态区
        for forest_type, eco_list in selected_ecos.items():
            if len(eco_list) < 2:
                continue

            # 获取这些生态区的气候信息
            eco_climates = {}
            for eco in eco_list:
                eco_climates[eco] = ecoregion_info[eco]['climate']

            # 检查是否有不同气候带的
            unique_climates = set(eco_climates.values())
            if len(unique_climates) < 2:
                print(f"森林类型 {forest_type} 的所有生态区气候带相同，跳过")
                continue

            print(f"\n森林类型 {forest_type} 迁移实验:")
            print(f"  生态区: {eco_list}")
            print(f"  气候带: {unique_climates}")

            # 相同森林类型内的所有迁移对
            for i in range(len(eco_list)):
                for j in range(len(eco_list)):
                    if i == j:
                        continue

                    source = eco_list[i]
                    target = eco_list[j]

                    # 只在不同气候带之间迁移
                    if eco_climates[source] == eco_climates[target]:
                        continue

                    print(f"  迁移: {source}({eco_climates[source]}) -> {target}({eco_climates[target]})")
                    result = self.migration_test(source, target)

                    if result:
                        result['experiment'] = 'Forest-type'
                        result['forest_type'] = forest_type
                        results.append(result)

        return pd.DataFrame(results)

    def run_all_experiments(self):
        """运行所有实验"""
        print("开始生态相似性迁移实验...")

        # 训练所有模型
        self.train_all_models()

        # 实验1：组内迁移
        df_exp1 = self.experiment_1_within_group_migration()

        # 实验2：跨组迁移
        df_exp2 = self.experiment_2_cross_group_migration()

        # 实验3：森林类型迁移
        df_exp3 = self.experiment_3_forest_type_migration()

        # 合并所有结果
        all_results = pd.concat([df_exp1, df_exp2, df_exp3], ignore_index=True)

        return all_results, df_exp1, df_exp2, df_exp3

    def analyze_results(self, df_exp1, df_exp2, df_exp3):
        """分析实验结果"""
        print("\n" + "="*80)
        print("实验结果分析")
        print("="*80)

        # 实验1分析：组内迁移
        if not df_exp1.empty:
            print("\n1. 组内迁移分析:")
            for group in df_exp1['group'].unique():
                df_group = df_exp1[df_exp1['group'] == group]
                avg_loss = df_group['performance_loss'].mean()
                avg_acc = df_group['migration_accuracy'].mean()
                print(f"  组 {group}: 平均性能损失={avg_loss:.1f}%, 平均迁移准确率={avg_acc:.3f}")

        # 实验2分析：跨组迁移
        if not df_exp2.empty:
            print("\n2. 跨组迁移分析:")
            # 按源-目标组分析
            cross_results = df_exp2.groupby(['source_group', 'target_group']).agg({
                'migration_accuracy': 'mean',
                'performance_loss': 'mean',
                'climate_similarity': 'mean'
            }).round(3)

            print(cross_results)

        # 实验3分析：森林类型迁移
        if not df_exp3.empty:
            print("\n3. 森林类型迁移分析:")
            for forest_type in df_exp3['forest_type'].unique():
                df_type = df_exp3[df_exp3['forest_type'] == forest_type]
                avg_loss = df_type['performance_loss'].mean()
                avg_acc = df_type['migration_accuracy'].mean()
                avg_sim = df_type['forest_type_similarity'].mean()
                print(f"  {forest_type}: 平均性能损失={avg_loss:.1f}%, 平均迁移准确率={avg_acc:.3f}, 类型相似度={avg_sim:.3f}")

        # 综合分析：性能损失与相似性的关系
        all_df = pd.concat([df_exp1, df_exp2, df_exp3], ignore_index=True)

        if not all_df.empty:
            print("\n4. 综合相关性分析:")
            # 计算相关性
            corr_loss_sim = all_df['performance_loss'].corr(all_df['climate_similarity'])
            corr_acc_sim = all_df['migration_accuracy'].corr(all_df['climate_similarity'])

            print(f"  性能损失与气候相似性的相关性: r = {corr_loss_sim:.3f}")
            print(f"  迁移准确率与气候相似性的相关性: r = {corr_acc_sim:.3f}")

            # 按实验类型分析
            print("\n5. 不同实验类型的平均性能:")
            exp_summary = all_df.groupby('experiment').agg({
                'migration_accuracy': ['mean', 'std'],
                'performance_loss': ['mean', 'std'],
                'local_accuracy': 'mean'
            }).round(3)

            print(exp_summary)

    def visualize_results(self, df_exp1, df_exp2, df_exp3):
        """可视化实验结果"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        # 1. 组内迁移性能损失
        if not df_exp1.empty:
            groups = df_exp1['group'].unique()
            group_loss = [df_exp1[df_exp1['group'] == g]['performance_loss'].mean() for g in groups]

            axes[0, 0].bar(groups, group_loss, color='skyblue')
            axes[0, 0].set_xlabel('气候组')
            axes[0, 0].set_ylabel('平均性能损失 (%)')
            axes[0, 0].set_title('组内迁移性能损失')
            axes[0, 0].grid(True, alpha=0.3)

        # 2. 跨组迁移矩阵
        if not df_exp2.empty:
            # 创建跨组迁移矩阵
            cross_matrix = pd.pivot_table(
                df_exp2,
                values='migration_accuracy',
                index='source_group',
                columns='target_group',
                aggfunc='mean'
            )

            sns.heatmap(cross_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
                       ax=axes[0, 1], cbar_kws={'label': '迁移准确率'})
            axes[0, 1].set_title('跨组迁移准确率矩阵')
            axes[0, 1].set_xlabel('目标组')
            axes[0, 1].set_ylabel('源组')

        # 3. 森林类型迁移性能
        if not df_exp3.empty:
            forest_types = df_exp3['forest_type'].unique()
            type_acc = [df_exp3[df_exp3['forest_type'] == t]['migration_accuracy'].mean() for t in forest_types]

            axes[0, 2].bar(forest_types, type_acc, color='lightgreen')
            axes[0, 2].set_xlabel('森林类型')
            axes[0, 2].set_ylabel('平均迁移准确率')
            axes[0, 2].set_title('森林类型迁移性能')
            axes[0, 2].tick_params(axis='x', rotation=45)
            axes[0, 2].grid(True, alpha=0.3)

        # 4. 性能损失与气候相似性关系
        all_df = pd.concat([df_exp1, df_exp2, df_exp3], ignore_index=True)
        if not all_df.empty:
            axes[1, 0].scatter(all_df['climate_similarity'], all_df['performance_loss'],
                              c='red', alpha=0.6, s=50)

            # 添加回归线
            if len(all_df) > 1:
                z = np.polyfit(all_df['climate_similarity'], all_df['performance_loss'], 1)
                p = np.poly1d(z)
                x_range = np.linspace(all_df['climate_similarity'].min(),
                                     all_df['climate_similarity'].max(), 100)
                axes[1, 0].plot(x_range, p(x_range), 'b--', alpha=0.7)

            axes[1, 0].set_xlabel('气候相似性')
            axes[1, 0].set_ylabel('性能损失 (%)')
            axes[1, 0].set_title('性能损失 vs 气候相似性')
            axes[1, 0].grid(True, alpha=0.3)

        # 5. 不同实验类型的性能对比
        if not all_df.empty:
            exp_types = all_df['experiment'].unique()
            exp_acc = [all_df[all_df['experiment'] == t]['migration_accuracy'].mean() for t in exp_types]
            exp_loss = [all_df[all_df['experiment'] == t]['performance_loss'].mean() for t in exp_types]

            x = np.arange(len(exp_types))
            width = 0.35

            bars1 = axes[1, 1].bar(x - width/2, exp_acc, width, label='迁移准确率', color='blue')
            bars2 = axes[1, 1].bar(x + width/2, exp_loss, width, label='性能损失', color='red')

            axes[1, 1].set_xlabel('实验类型')
            axes[1, 1].set_ylabel('值')
            axes[1, 1].set_title('不同实验类型性能对比')
            axes[1, 1].set_xticks(x)
            axes[1, 1].set_xticklabels(exp_types)
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)

        # 6. 文本总结
        axes[1, 2].axis('off')
        if not all_df.empty:
            summary_text = f"""
生态相似性迁移实验总结

总迁移实验数: {len(all_df)}
平均迁移准确率: {all_df['migration_accuracy'].mean():.3f}
平均性能损失: {all_df['performance_loss'].mean():.1f}%

关键发现:
1. 组内迁移损失最小: {df_exp1['performance_loss'].mean():.1f}%
2. 跨组迁移损失最大: {df_exp2['performance_loss'].mean():.1f}%
3. 气候相似性与性能损失相关性: {all_df['performance_loss'].corr(all_df['climate_similarity']):.3f}

结论:
• 生态相似性显著影响模型迁移性能
• ECF-TST框架在相似生态条件下具有良好的可迁移性
• 在差异较大区域需要独立建模
"""
            axes[1, 2].text(0.1, 0.5, summary_text, transform=axes[1, 2].transAxes,
                           fontsize=11, verticalalignment='center',
                           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

        plt.suptitle('生态相似性迁移实验综合分析', fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.show()

        # 保存图表
        plt.savefig('/content/drive/MyDrive/ECF_TST_Results/migration_analysis.png',
                   dpi=300, bbox_inches='tight')
        print("✓ 可视化结果已保存")

# ==================== 运行迁移实验 ====================
print("准备运行生态相似性迁移实验...")

# 加载数据（假设数据已经加载）
# df = 您的数据框
# feature_cols = 您的特征列

# 创建实验对象
experiment = EcoMigrationExperiment(df, feature_cols)

# 运行所有实验
all_results, df_exp1, df_exp2, df_exp3 = experiment.run_all_experiments()

# 分析结果
experiment.analyze_results(df_exp1, df_exp2, df_exp3)

# 可视化结果
experiment.visualize_results(df_exp1, df_exp2, df_exp3)

# 保存结果
print("\n保存实验结果...")
output_path = '/content/drive/MyDrive/ECF_TST_Results'
all_results.to_csv(f'{output_path}/migration_experiment_results.csv', index=False)
df_exp1.to_csv(f'{output_path}/within_group_migration.csv', index=False)
df_exp2.to_csv(f'{output_path}/cross_group_migration.csv', index=False)
df_exp3.to_csv(f'{output_path}/forest_type_migration.csv', index=False)

print("✓ 所有实验完成！")
print(f"✓ 结果已保存到: {output_path}")

生态相似性迁移实验
准备运行生态相似性迁移实验...


NameError: name 'df' is not defined